<a href="https://www.kaggle.com/code/hatemamine/fine-tuning-biencoder-using-knowledge-distilation?scriptVersionId=222423161" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

%env PATH=/usr/local/cuda/bin:/opt/conda/bin:/opt/conda/condabin:/opt/conda/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin!pip install huggingface_hub -q!pip install datasets -q!pip install torch -q!pip install transformers -q!pip install tensorflow-cpu==2.16.1 -q!pip install tf-keras==2.16.0 --no-dependencies -q!pip install wandb -q!pip install -U accelerate -q!pip install -U transformers[torch]==4.45.2 -q!pip install sentence_transformers==3.1.1 -q

In [ ]:
organization_dataset_id="hatemestinbejaia/ExperimentDATA_knowledge_distillation_vs_fine_tuning"
#model ="mmarco-Arabic-AraElectra-bi-encoder-NoKD-v1"
#model ="mmarco-Arabic-AraDPR-bi-encoder-NoKD-v1"
#model ="mmarco-Arabic-mMiniLML-bi-encoder-NoKD-v1"
# fine-tuning using knowledge distilation
#model ="mmarco-Arabic-AraElectra-bi-encoder-KD-v1"
#model ="mmarco-Arabic-AraDPR-bi-encoder-KD-v1"
model ="mmarco-Arabic-mMiniLML-bi-encoder-KD-v1"
dataset_name= "KD_train_dataset_t05M_t15m_eval_10k_preprocessedAraELECTRA"
dev_name= "dev_samples_araelectra.pkl"
if model =="mmarco-Arabic-AraElectra-bi-encoder-KD-v1":    
    args_model="aubmindlab/araelectra-base-discriminator"    
if model =="mmarco-Arabic-AraElectra-bi-encoder-NoKD-v1" :    
    args_model="aubmindlab/araelectra-base-discriminator"    
if model =="mmarco-Arabic-AraDPR-bi-encoder-NoKD-v1" :    
    args_model="abdoelsayed/AraDPR"    
if model =="mmarco-Arabic-AraDPR-bi-encoder-KD-v1" :    
    args_model="abdoelsayed/AraDPR"    
if model =="mmarco-Arabic-mMiniLML-bi-encoder-NoKD-v1" :    
    args_model="sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2"    
if model =="mmarco-Arabic-mMiniLML-bi-encoder-KD-v1" :    
    args_model="sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2" 
checkpoint =model

In [ ]:
!pip install sentence_transformers -q
!pip install huggingface_hub -q
!pip install wandb -q

In [ ]:
#get the train dataset
from datasets import load_from_disk, load_dataset
train_dataset = load_dataset(organization_dataset_id, dataset_name, trust_remote_code=True)

In [ ]:
train_dataset = train_dataset.remove_columns(['pos_score', 'neg_score'])
print(train_dataset)
train_dataset["train0"][0]

In [ ]:
import torch
from datasets import load_dataset
from datetime  import datetime
from sentence_transformers import SentenceTransformer, SentenceTransformerTrainer, losses
from sentence_transformers.training_args import SentenceTransformerTrainingArguments
from datasets import Dataset
from sentence_transformers.evaluation import TripletEvaluator, RerankingEvaluator
from transformers import EarlyStoppingCallback

In [ ]:
from transformers import AutoTokenizer, AutoModel
tokenizer  = AutoTokenizer.from_pretrained(args_model)
model = SentenceTransformer(args_model)

In [ ]:
# to use wandb uncomment and change report_to = 'wandb' in TrainingArguments
"""
import wandb
import os
os.environ["WANDB_API_KEY"] = "your wandb api key"
"""

In [ ]:
import accelerate
import transformers
transformers.__version__, accelerate.__version__
device = "cuda:0" if torch.cuda.is_available() else "cpu"
print(device)

In [ ]:
eval_steps =2000
from transformers import EarlyStoppingCallback, IntervalStrategy
training_args = SentenceTransformerTrainingArguments(        
    f"training_with_callbacks",        
    evaluation_strategy = IntervalStrategy.STEPS, # "steps"        
    eval_steps = eval_steps, # Evaluation and Save happens every 50 steps        
    save_total_limit = 2,        
    save_steps= eval_steps,        
    fp16=True,        
    fp16_backend="amp",        
    per_device_train_batch_size=16,         
    gradient_accumulation_steps=8,        
    logging_steps=eval_steps,        
    warmup_ratio=0.07,        
    num_train_epochs=3,         
    learning_rate = 7e-5,        
    weight_decay= 0.0,        
    metric_for_best_model = 'eval_mrr@10',        
    greater_is_better=True,       
    load_best_model_at_end=True,       
    report_to = None,             # report_to = 'wandb' enable logging to W&B       
    run_name = checkpoint            # name of the W&B run    
   )

In [ ]:
import torch
#get the eval dataset
from huggingface_hub import hf_hub_download
hf_hub_download(repo_id=organization_dataset_id, filename=dev_name, local_dir="./", repo_type="dataset")
import pickle
from sentence_transformers.cross_encoder.evaluation  import CERerankingEvaluator
with open(dev_name, 'rb') as f : dev_samples = pickle.load(f)
RerankingEvaluator(dev_samples)
#results = triplet_evaluator(student_model)# In this example, the labels become -0.036 and 0.68, respectively
#loss = losses.MarginMSELoss(model)
#train_loss = losses.MultipleNegativesRankingLoss(model=model)

In [ ]:
from __future__ import annotations
from sklearn.preprocessing import normalize
from collections.abc import Iterable

from torch import Tensor, nn
import torch.nn.functional as F

from sentence_transformers import SentenceTransformer, util


class MarginMSELoss(nn.Module):
    def __init__(self, model: SentenceTransformer, similarity_fct=util.pairwise_dot_score) -> None:
        """
        Compute the MSE loss between the ``|sim(Query, Pos) - sim(Query, Neg)|`` and ``|gold_sim(Query, Pos) - gold_sim(Query, Neg)|``.
        By default, sim() is the dot-product. The gold_sim is often the similarity score from a teacher model.

        In contrast to :class:`MultipleNegativesRankingLoss`, the two passages do not have to be strictly positive and negative,
        both can be relevant or not relevant for a given query. This can be an advantage of MarginMSELoss over
        MultipleNegativesRankingLoss, but note that the MarginMSELoss is much slower to train. With MultipleNegativesRankingLoss,
        with a batch size of 64, we compare one query against 128 passages. With MarginMSELoss, we compare a query only
        against two passages.

        Args:
            model: SentenceTransformerModel
            similarity_fct: Which similarity function to use.

        References:
            - For more details, please refer to https://arxiv.org/abs/2010.02666.
            - `Training Examples > MS MARCO <../../examples/training/ms_marco/README.html>`_
            - `Unsupervised Learning > Domain Adaptation <../../examples/domain_adaptation/README.html>`_

        Requirements:
            1. (query, passage_one, passage_two) triplets
            2. Usually used with a finetuned teacher M in a knowledge distillation setup

        Inputs:
            +-----------------------------------------------+-----------------------------------------------+
            | Texts                                         | Labels                                        |
            +===============================================+===============================================+
            | (query, passage_one, passage_two) triplets    | M(query, passage_one) - M(query, passage_two) |
            +-----------------------------------------------+-----------------------------------------------+

        Relations:
            - :class:`MSELoss` is equivalent to this loss, but without a margin through the negative pair.

        Example:

            With gold labels, e.g. if you have hard scores for sentences. Imagine you want a model to embed sentences
            with similar "quality" close to each other. If the "text1" has quality 5 out of 5, "text2" has quality
            1 out of 5, and "text3" has quality 3 out of 5, then the similarity of a pair can be defined as the
            difference of the quality scores. So, the similarity between "text1" and "text2" is 4, and the
            similarity between "text1" and "text3" is 2. If we use this as our "Teacher Model", the label becomes
            similraity("text1", "text2") - similarity("text1", "text3") = 4 - 2 = 2.

            Positive values denote that the first passage is more similar to the query than the second passage,
            while negative values denote the opposite.

            ::

                from sentence_transformers import SentenceTransformer, SentenceTransformerTrainer, losses
                from datasets import Dataset

                model = SentenceTransformer("microsoft/mpnet-base")
                train_dataset = Dataset.from_dict({
                    "text1": ["It's nice weather outside today.", "He drove to work."],
                    "text2": ["It's so sunny.", "He took the car to work."],
                    "text3": ["It's very sunny.", "She walked to the store."],
                    "label": [0.1, 0.8],
                })
                loss = losses.MarginMSELoss(model)

                trainer = SentenceTransformerTrainer(
                    model=model,
                    train_dataset=train_dataset,
                    loss=loss,
                )
                trainer.train()

            We can also use a teacher model to compute the similarity scores. In this case, we can use the teacher model
            to compute the similarity scores and use them as the silver labels. This is often used in knowledge distillation.

            ::

                from sentence_transformers import SentenceTransformer, SentenceTransformerTrainer, losses
                from datasets import Dataset

                student_model = SentenceTransformer("microsoft/mpnet-base")
                teacher_model = SentenceTransformer("all-mpnet-base-v2")
                train_dataset = Dataset.from_dict({
                    "query": ["It's nice weather outside today.", "He drove to work."],
                    "passage1": ["It's so sunny.", "He took the car to work."],
                    "passage2": ["It's very sunny.", "She walked to the store."],
                })

                def compute_labels(batch):
                    emb_queries = teacher_model.encode(batch["query"])
                    emb_passages1 = teacher_model.encode(batch["passage1"])
                    emb_passages2 = teacher_model.encode(batch["passage2"])
                    return {
                        "label": teacher_model.similarity_pairwise(emb_queries, emb_passages1) - teacher_model.similarity_pairwise(emb_queries, emb_passages2)
                    }

                train_dataset = train_dataset.map(compute_labels, batched=True)
                # In this example, the labels become -0.036 and 0.68, respectively
                loss = losses.MarginMSELoss(student_model)

                trainer = SentenceTransformerTrainer(
                    model=student_model,
                    train_dataset=train_dataset,
                    loss=loss,
                )
                trainer.train()
        """
        super().__init__()
        self.model = model
        self.similarity_fct = similarity_fct
        self.loss_fct = nn.MSELoss()

    def forward(self, sentence_features: Iterable[dict[str, Tensor]], labels: Tensor) -> Tensor:
        # sentence_features: query, positive passage, negative passage
        reps = [self.model(sentence_feature)["sentence_embedding"] for sentence_feature in sentence_features]
        embeddings_query = reps[0]
        embeddings_pos = reps[1]
        embeddings_neg = reps[2]

        scores_pos = self.similarity_fct(embeddings_query, embeddings_pos)
        scores_neg = self.similarity_fct(embeddings_query, embeddings_neg)
        margin_pred = scores_pos - scores_neg
        #print(margin_pred.shape)
        #print(margin_pred)
        margin_pred= F.normalize(margin_pred, p=1.0, dim = 0)
        labels= F.normalize(labels, p=1.0, dim = 0)
        return self.loss_fct(margin_pred, labels)

    @property
    def citation(self) -> str:
        return """
@misc{hofstätter2021improving,
    title={Improving Efficient Neural Ranking Models with Cross-Architecture Knowledge Distillation},
    author={Sebastian Hofstätter and Sophia Althammer and Michael Schröder and Mete Sertkan and Allan Hanbury},
    year={2021},
    eprint={2010.02666},
    archivePrefix={arXiv},
    primaryClass={cs.IR}
}
"""
loss = MarginMSELoss(model)

In [ ]:
from transformers import TrainerCallback
class MyCallback(TrainerCallback):
    "A callback that prints a message at the beginning of training"

    def on_save(self, args, state, control, **kwargs):
        """
        Event called after a checkpoint save.
        """
        print("save checkpoint")
        print(state.global_step)
        if state.global_step ==-1 : control.should_training_stop=True

In [ ]:
# 6. Create the trainer & start training
trainer = SentenceTransformerTrainer(    
    model=model,    
    args=training_args,    
    train_dataset=train_dataset["train0"],    
    eval_dataset= train_dataset["test"],    
    loss=loss,    
    evaluator=RerankingEvaluator(dev_samples),    
    tokenizer=tokenizer,    
    callbacks = [EarlyStoppingCallback(early_stopping_patience=10), MyCallback]    
    #evaluator=triplet_evaluator,
    )
#train_result = trainer.train()


In [ ]:
trainer.train()
#trainer.train(resume_from_checkpoint = True)
#trainer.save_model()

In [ ]:
# uncomment to push to yrour huggingface hub
"""
import huggingface_hub 
hf = huggingface_hub.HfFolder()
access_token = "your access token"
hf.save_token(access_token)
model.push_to_hub("hatemestinbejaia/"+checkpoint)
tokenizer.push_to_hub("hatemestinbejaia/"+checkpoint)
"""

In [ ]:
testmodel = SentenceTransformer("hatemestinbejaia/"+checkpoint)
dev_evaluator=RerankingEvaluator(dev_samples)
result= dev_evaluator(testmodel)
result